Chrome is a memory hog, this notebook works best in other browsers.

In [1]:
import pickle
import sys
sys.path.append('python')
import clusterOutliers
with open('Q8_sample.coo','rb') as file:
    Q8_coo = pickle.load(file)

In [76]:
import numpy as np
from quarterTools import read_kepler_curve, pca_red
import pandas as pd

data = Q8_coo.data
Quarter = 8

pca=pca_red(data)
x = pca.iloc[:,0]
y = pca.iloc[:,1]
files = data.index
labels = np.array(['KIC '+str(int(lc[4:13])) for lc in files])

Scaling data using StandardScaler...
Finding minimum number of dimensions to explain 90.0% of the variance...

        Dimensions: 16,
        Variance explained: 90.2%
        


In [4]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [89]:

from bokeh.layouts import row,gridplot,column
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Button
from bokeh.events import Tap
from lightkurve import search_lightcurvefile

def modify_doc(doc):
    TOOLS="pan,wheel_zoom,reset,tap,box_select,poly_select"
    quarter = Quarter
    # create a column data source for the plots to share
    s1 = ColumnDataSource(data={
        'x'     : x,
        'y'     : y,
        'desc'  : labels,
        'files' : files
    }) 

    s2=ColumnDataSource(data={
        't'  : [],
        'nf' : []
    })

    # Set up widgets
    button = Button(label='Plot Selected')
    # Set up callbacks
    def update_plot():
        try:
            inds = [s1.selected.indices[0]] # at present, plotting multiple light curves isn't supported
        except:
            inds = [0]
        lcs = list(s1.data['desc'][inds])
        for i,ind in enumerate(inds):
            lc = lcs[i]
            
            # download Kepler lighcurve via Lightkurve
            lcf = search_lightcurvefile(lc, mission="Kepler", quarter=quarter).download()
            # use the normalized PDCSAP flux 
            nlc = lcf.PDCSAP_FLUX.normalize()
            t = nlc.time
            nf = nlc.flux
            
            s2.data = {'t':t,'nf':nf}
            plc.line('t','nf',source=s2)
        
    button.on_click(update_plot)

    # Set up layouts and add to document
    inputs = column(button)

    # create a new plot and add a renderer
    left = figure(tools=TOOLS, plot_width=1000, plot_height=600, title=None)
    left.circle('x', 'y', source=s1)
    left.on_event(Tap,update_plot)
    # Planning to incorporate a detailed view of the cluster center on a right plot in the future
    # create a plot for the light curve and add a line renderer
    plc = figure(tools=TOOLS,plot_width=1000,plot_height=200,title=None)
    plc.line('t','nf',source=s2)
    update_plot()

    #p = gridplot([[left,right]]) #future planning
    layout = column(inputs,left, plc)
    doc.add_root(layout)
    
show(modify_doc,notebook_url="http://localhost:8888")